In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e2/sample_submission.csv
/kaggle/input/playground-series-s5e2/train.csv
/kaggle/input/playground-series-s5e2/test.csv
/kaggle/input/playground-series-s5e2/training_extra.csv


In [2]:
train=pd.read_csv('/kaggle/input/playground-series-s5e2/train.csv')
test=pd.read_csv('/kaggle/input/playground-series-s5e2/test.csv')

train=train.drop(['id'],axis=1)
#test=test.drop(['id'],axis=1)

In [ ]:
train.columns[train.dtypes=='object']

In [ ]:
train[train['Weight Capacity (kg)']=='Nan']

In [ ]:
train['Brand'].unique()

In [3]:
train = train.replace(np.nan, "Na")


In [ ]:
train['Weight Capacity (kg)']=train['Weight Capacity (kg)'].replace("Nan",np.nan)

In [4]:
import catboost
from catboost import CatBoostRegressor
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


In [5]:
X=train.drop(['Price'],axis=1)
y=train['Price']

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)

In [ ]:
# Initialize CatBoost Regressor
catboost_model = CatBoostRegressor(
    iterations=500,  # Number of boosting iterations
    learning_rate=0.1,  # Step size shrinkage
    depth=6,  # Maximum depth of trees
    loss_function='RMSE',  # Root Mean Squared Error for regression
    verbose=100,
    cat_features=['Brand', 'Material', 'Size', 'Laptop Compartment', 'Waterproof',
       'Style', 'Color']# Print training progress every 100 iterations
)

# Train the model
catboost_model.fit(X_train, y_train)


In [ ]:
preds=catboost_model.predict(X_test)

In [ ]:
mean_squared_error(preds,y_test,squared=False)

In [ ]:
test = test.replace(np.nan, "Na")

In [ ]:
preds1=catboost_model.predict(test.drop(['id'],axis=1))

In [ ]:
preds1

In [6]:
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV

In [7]:
param_dist = {
    'iterations': [500, 1000, 1500],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'depth': [4, 6, 8, 10],
    'l2_leaf_reg': [1, 3, 5, 7],
    'subsample': [0.6, 0.8, 1.0]
}

# Initialize CatBoost model
catboost_model = CatBoostRegressor(loss_function="RMSE", verbose=100,cat_features=['Brand', 'Material', 'Size', 'Laptop Compartment', 'Waterproof',
       'Style', 'Color'])

# Use RandomizedSearchCV
random_search = RandomizedSearchCV(
    catboost_model, param_distributions=param_dist, n_iter=10, cv=3, scoring="neg_mean_squared_error", random_state=42
)
random_search.fit(X_train, y_train)

0:	learn: 39.0740875	total: 170ms	remaining: 2m 49s
100:	learn: 39.0106074	total: 6.04s	remaining: 53.8s
200:	learn: 38.9684872	total: 13.1s	remaining: 52.2s
300:	learn: 38.9222851	total: 20.7s	remaining: 48s
400:	learn: 38.8843913	total: 28.7s	remaining: 42.9s
500:	learn: 38.8423279	total: 36.8s	remaining: 36.7s
600:	learn: 38.8044861	total: 44.6s	remaining: 29.6s
700:	learn: 38.7683009	total: 52.8s	remaining: 22.5s
800:	learn: 38.7285618	total: 1m 1s	remaining: 15.3s
900:	learn: 38.6917889	total: 1m 9s	remaining: 7.67s
999:	learn: 38.6526050	total: 1m 18s	remaining: 0us
0:	learn: 38.9733768	total: 49.9ms	remaining: 49.9s
100:	learn: 38.9158664	total: 5.96s	remaining: 53.1s
200:	learn: 38.8770414	total: 13.7s	remaining: 54.6s
300:	learn: 38.8365117	total: 21.3s	remaining: 49.5s
400:	learn: 38.7991445	total: 29.2s	remaining: 43.6s
500:	learn: 38.7581884	total: 37s	remaining: 36.9s
600:	learn: 38.7177601	total: 45.7s	remaining: 30.3s
700:	learn: 38.6766362	total: 53.9s	remaining: 23s
80

RandomizedSearchCV(cv=3,
                   estimator=<catboost.core.CatBoostRegressor object at 0x782054752560>,
                   param_distributions={'depth': [4, 6, 8, 10],
                                        'iterations': [500, 1000, 1500],
                                        'l2_leaf_reg': [1, 3, 5, 7],
                                        'learning_rate': [0.01, 0.05, 0.1, 0.2],
                                        'subsample': [0.6, 0.8, 1.0]},
                   random_state=42, scoring='neg_mean_squared_error')

In [ ]:
 random_search.best_params_

In [ ]:
best_params = random_search.best_params_  # Or grid_search.best_params_

# Train the final model
final_model = CatBoostRegressor(**best_params, verbose=100)
final_model.fit(X_train, y_train)

# Evaluate
y_pred = final_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"Final Model RMSE: {rmse:.4f}")


In [ ]:
submission_df = pd.DataFrame({
    'id': test['id'],
    'Price':preds1
})

# Save to CSV
submission_df.to_csv("submission.csv", index=False)

submission_df.head(5)